In [1]:
import pandas as pd 
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')



display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [2]:
import nltk 
import numpy as np
import re
import string

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import files
path="https://github.com/hazmohd96/i-CR-Project/raw/main/db(newcategory).xlsx"
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from tabulate import tabulate
from textblob import TextBlob
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer


df_action= pd.read_excel(path,sheet_name='Action Items')
df_issue= pd.read_excel(path,sheet_name='Issues')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
stop = stopwords.words('english')
lookup_dict = {'eg':'example','etc':'et cetera','hse':'health, safety and environment,','csr':'corporate social responsibility', 'wbu' : 'what about you'}

def text_std(input_text):
  words = input_text.split()
  for word in words:
    if word.lower() in lookup_dict:
      word = lookup_dict[word.lower()]
  return word
  
def vectorize():
  frames =[df_issue['lemmatizing'],df_action['lemmatizing']]
  new_df =pd.concat(frames)
  vectorizer= TfidfVectorizer()
  new_vectorize=vectorizer.fit_transform(new_df)
  new_issue_vec= new_vectorize[0:df_issue['lemmatizing'].shape[0]]
  new_action_vec= new_vectorize[df_issue['lemmatizing'].shape[0]:new_vectorize.shape[0]]
  return[new_issue_vec,new_action_vec]

df_issue['Project name']= df_issue['Project name'].map(lambda x: x.upper())
df_issue['Category']= df_issue['Category'].map(lambda x: x.upper())

df_action['Project name']= df_action['Project name'].map(lambda x: x.upper())
df_action['Category']= df_action['Category'].map(lambda x: x.upper())



#filtering the database
category_list=df_issue['Category'].drop_duplicates()
print(category_list.reset_index(drop=True))

category_input =(input("\nEnter Category \n")).upper()
sub2_proj =df_issue.loc[df_issue['Category']==category_input]
sub2_proj1 =df_action.loc[df_action['Category']==category_input]


ndf_issue = sub2_proj.copy()
ndf_action=sub2_proj1.copy()
ndf_issue = ndf_issue.reset_index(drop=True)
ndf_action = ndf_action.reset_index(drop=True)

#text processing issue
ndf_issue['lower']= ndf_issue['Issues'].map(lambda x: x.lower())
ndf_issue['punct']= ndf_issue['lower'].str.replace('[^\w\s]','')
ndf_issue['stop']= ndf_issue['punct'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
ndf_issue['standard']=ndf_issue['stop'][:500].apply(lambda x: " ".join([text_std(word) for word in x.split()]))
ndf_issue['correct']=ndf_issue['standard'].apply(lambda x: str(TextBlob(x).correct()))
ndf_issue['lemmatizing'] =ndf_issue['correct'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

#text processing action
ndf_action['lower']= ndf_action['Action Items'].map(lambda x: x.lower())
ndf_action['punct']=ndf_action['lower'].str.replace('[^\w\s]','')
ndf_action['stop']= ndf_action['punct'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
ndf_action['standard']=ndf_action['stop'][:500].apply(lambda x: " ".join([text_std(word) for word in x.split()]))
ndf_action['correct']=ndf_action['standard'].apply(lambda x: str(TextBlob(x).correct()))
ndf_action['lemmatizing'] = ndf_action['correct'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

_action_vec=""
issue_option=""

def vectorize():
  frames =[ndf_issue['lemmatizing'],ndf_action['lemmatizing']]
  new_df =pd.concat(frames)
  vectorizer= TfidfVectorizer()
  new_vectorize=vectorizer.fit_transform(new_df)
  new_issue_vec= new_vectorize[0:ndf_issue['lemmatizing'].shape[0]]
  new_action_vec= new_vectorize[ndf_issue['lemmatizing'].shape[0]:new_vectorize.shape[0]]
  return[new_issue_vec,new_action_vec]

user_input = input("Enter new issue =1, select existing issues =2 ")

if user_input == "1":
  text=input("Enter new issue : ")
  ndf_issue= ndf_issue.append({'ISSUES':text},ignore_index=True ) 
  ndf_issue['lower']= ndf_issue.map(lambda x: x.lower())
  ndf_issue['punct']=ndf_issue['lower'].str.replace('[^\w\s]','')
  ndf_issue['stop']= ndf_issue['punct'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  ndf_issue['standard']=ndf_issue['stop'][:100].apply(lambda x: " ".join([text_std(word) for word in x.split()]))
  ndf_issue['correct']=ndf_issue['standard'].apply(lambda x: str(TextBlob(x).correct()))
  ndf_issue['lemmatizing'] = ndf_issue['correct'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  new_issue_vec,new_action_vec=vectorize()
  issue_option = new_issue_vec[-1]
  print("\nSelected Issue : ",text)

elif user_input=="2" :
  display(ndf_issue['Issues'])
  text2=int(input("Select issue number :"))
  print("\n\nSelected Issue :",ndf_issue['Issues'][text2])
  new_issue_vec,new_action_vec=vectorize()
  issue_option = new_issue_vec[text2]

c_similarity_Action = cosine_similarity(new_action_vec,issue_option)
b= c_similarity_Action.tolist()
b.sort(reverse=True)
ndf_action['Cosine']= c_similarity_Action
ndf_result = ndf_action[['Cosine','Action Items']]
ndf_rduplicate=(ndf_result.drop_duplicates())
#cosine_result = df_action.loc[df_action['Cosine']>0.0]
sort= ndf_rduplicate.sort_values(by='Cosine', ascending=False).head(20)
display(sort)

if user_input =='1':
  df_issue=df_issue[:-1]



0                                  PLOT PLAN, LAYOUT & HEAVY LIFT
1                                            TEMPORARY FACILITIES
2                                                          OTHERS
3                                     CIVIL, STRUCTURE & BUILDING
4                                                STATIC EQUIPMENT
5                                                          PIPING
6                                                      ELECTRICAL
7                                                      INSTRUMENT
8                                                STATIC EQUIPMENT
9                                                   PACKAGED UNIT
10                                    CIVIL, STRUCTURE & BUILDING
11                          SCHEDULE IMPACT DUE TO TA DATE CHANGE
12                                               ADVERSE WEATHER 
13                           EMERGENCY READINESS SUPPORTING PLANT
14                                       THEFT IN PLANT DURING TA
15        

<ipython-input-5-c61d26bda7e7>:44: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf_issue['punct']= ndf_issue['lower'].str.replace('[^\w\s]','')
<ipython-input-5-c61d26bda7e7>:52: FutureWarning: The default value of regex will change from True to False in a future version.
  ndf_action['punct']=ndf_action['lower'].str.replace('[^\w\s]','')


Enter new issue =1, select existing issues =2 2


0                                                                          Noticed the Raised Floors in the instrument room of Control Building are just supported by planks (near cable entry area). This is not accetable and it's a HSE concern.
1                                                      There will be 2 new filter vessels of larger capacity and size than existing ones. Due to the MS is situated at "cut ground" area, there may not be any piling but just large pad footings. 
2                 There are existing underground facilities, i.e. cables, pipes etc. as indicated in As-Built drawings. Some may be identified and some are not. These facilities may be damaged due to excavation or heavy vehicles running over. 
3                                                                                                                                                       Due to current "narrow" road to MS, there is a concern of heavy vehicle access, i.e. crane.
4     For replacing of m

Select issue number :11


Selected Issue : Location of strength jack support beam yet to be identified 


,Cosine,Action Items
17,0.202375,To ensure location is clash-free
29,0.103767,Survey for new equipment location and soil investigation need to be conducted to verify soil parameters and to identify existing facilities.
25,0.084859,"To confirm on the capacity of raised floor to cater the additional DB, otherwise a separate steel structure frame is needed to support the DB."
0,0.000000,Need to find other permenant supporting arrangement.
44,0.000000,EPCC to conduct underground scanning and pilot holes to physically verify the underground scanning report from Owner
34,0.000000,To carry structural asessment on the existing pipe racks/supports.
35,0.000000,To establish blinding plan for pipe isolation (for tie-in works).
36,0.000000,PMT to engage with GDC Operation to establish PTW Application and Approval Plan which will be included in BEPCC ITB.
37,0.000000,To carry out soil investigation and to establish current water level
38,0.000000,To determine the correct construction method
